In [1]:
# 유투브에서 검색 후 동영상의 리뷰 정보 수집하기
#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver

import time
import sys
import re
import math
import numpy 
import pandas as pd   
import xlwt 
import random
import os


#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.
print("=" *80)
print("유투브 영상의 댓글을 수집하는 웹크롤러입니다")
print("=" *80)

query_txt=input("1.유튜브에서 검색할 주제 키워드를 입력하세요(예:올리브영): ")

cnt = int(input('2.위 주제로 댓글을 크롤링할 유튜브 영상은 몇건입니까?:'))
reple_cnt = int(input('3.각 동영상에서 추출할 댓글은 몇건입니까?: '))

f_dir = input("4.크롤링 결과를 저장할 폴더명만 쓰세요(기본값:c:\\temp\\):")
if f_dir == '' :
    f_dir = 'c:\\temp\\'

print("\n")
print("요청하신 정보로 데이터를 수집중이니 잠시만 기다려 주세요~~^^")   

 # Step 3. 저장될 파일위치와 이름을 지정 한 후 크롬 드라이버를 실행하여 페이지를 엽니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+'youtube'+'-'+s+'-'+query_txt)
os.chdir(f_dir+'youtube'+'-'+s+'-'+query_txt)

ff_dir=f_dir+'youtube'+'-'+s+'-'+query_txt
ff_name=f_dir+'youtube'+'-'+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+'youtube'+'-'+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+'youtube'+'-'+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'

s_time = time.time( )

# Step 2. 크롬 드라이버 설정 
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
driver= webdriver.Chrome()

driver.get('https://www.youtube.com')

time.sleep(5)
element = driver.find_element_by_name("search_query")
element.send_keys(query_txt)
element.submit()
time.sleep(2)

#검색 결과가 첫 화면에 20개가 나옵니다.
# 만약 사용자가 입력한 동영상 개수가 20개가 넘어갈 경우 스크롤다운해서 갯수를
# 먼저 만든 후 URL 을 추출하여 URL 을 완성해야 합니다

def scroll_down(driver):
    #driver.execute_script("window.scrollTo(500,document.body.scrollHeight);")
    driver.execute_script("window.scrollBy(0,3000);") # 한페이지 20개씩 출력값
    time.sleep(5)

if reple_cnt < 20 :
    page_cnt = 1 
else :
    page_cnt = math.ceil(cnt/20)

if cnt > 20 :
    i = 1
    while ( i <=  page_cnt) :
        print("화면을 %s 회 아래로 스크롤 합니다" %i)
        scroll_down(driver)
        time.sleep(1)
        i += 1

# 검색된 유튜브 영상의 URL 을 추출합니다

time.sleep(2)

html = driver.page_source
soup1 = BeautifulSoup(html, 'html.parser')

count = 0
item=[]

print("\n")

for i in soup1.find_all('a','yt-simple-endpoint style-scope ytd-video-renderer'):
    item.append(i['href'])
    count += 1

    if count == cnt :
        break

print("검색된 영상 중 %s 건 동영상의 댓글을 추출하겠습니다" %cnt)
print("\n")

# 비트맵 이미지 아이콘을 위한 대체 딕셔너리를 만든다
bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

# 수집된 전체 URL 중에서 사용자가 입력한 수만큼의 동영상의 댓글을 수집합니다.
# 먼저 전체 URL 을 완성 -> 동영상 페이지 열기 -> 댓글 수집 -> 다음 동영상 순으로 진행합니다
full_url=[]
url_cnt = 0
for x in range(0,len(item)) :
    url_cnt += 1
    url = 'https://www.youtube.com/' +item[x]
    full_url.append(url)
    if url_cnt == cnt :
        break

play_cnt = 1

url2=[]
reple2=[]
reple3=[]
reple4=[]
writer2=[]
wdate2=[]

for y in range(0,len(full_url)) :
    driver.get(full_url[y])
    time.sleep(10)
    print("\n")
    print("\n")
    print("%s 번째 동영상의 정보를 수집합니다." %play_cnt)

    i = 1
    while ( i <=  page_cnt) :
        scroll_down(driver)
        time.sleep(5)
        i += 1

    html = driver.page_source
    soup2 = BeautifulSoup(html, 'html.parser')

    t_count_0 = soup2.select('#count')
    t_count_1 = t_count_0[1].get_text()
    t_count_2 = t_count_1.replace(",","")
    t_count_3 = re.search("\d+",t_count_2)
    t_count_4 = int(t_count_3.group())

    t_title_1 = soup2.select('#info-contents')
    t_title_2 = t_title_1[0].find('h1').get_text()
    t_title_3 = t_title_2.translate(bmp_map).replace("\n","")

    t_view_1 = soup2.find('yt-formatted-string','count-text style-scope ytd-comments-header-renderer').get_text()
    t_view_2 = t_view_1.replace("," ,"")
    t_view_3 = re.search("\d+",t_view_2)
    print(t_view_1)
    t_view_4 = int(t_view_3.group())

    print("=" *80)
    print("%s 번째 동영상의 조회수는 %s 회 이고 수집 가능한 댓글수는 총 %s개 입니다" %(play_cnt,t_count_4,t_view_4))  
    print("%s 번째 동영상의 제목은 %s 입니다" %(play_cnt,t_title_3))
    print("=" *80)
    print("\n")
    
    
    play_cnt += 1
    

유투브 영상의 댓글을 수집하는 웹크롤러입니다
1.유튜브에서 검색할 주제 키워드를 입력하세요(예:올리브영): 올리브영
2.위 주제로 댓글을 크롤링할 유튜브 영상은 몇건입니까?:10
3.각 동영상에서 추출할 댓글은 몇건입니까?: 5
4.크롤링 결과를 저장할 폴더명만 쓰세요(기본값:c:\temp\):


요청하신 정보로 데이터를 수집중이니 잠시만 기다려 주세요~~^^


검색된 영상 중 10 건 동영상의 댓글을 추출하겠습니다






1 번째 동영상의 정보를 수집합니다.


AttributeError: 'NoneType' object has no attribute 'get_text'